# トピックモデルによるクラスタリング
トピックモデルとは文書中の単語出現分布を元に傾向（≒トピックらしきもの）を観察しようとするアプローチで、クラスタリングの一種である。なお、一般的なクラスタリング（例えば[k平均法](https://ja.wikipedia.org/wiki/K平均法)）では一つのサンプルが一つのクラスタに属するという前提でグルーピングを行うのに対し、トピックモデルでは一つのサンプルが複数のクラスタを内包しているという前提でグルーピングを行う。次の例を眺めるとイメージをつかみやすいだろう。

- 例1: [トピックモデル入門：WikipediaをLDAモデル化してみた](https://recruit.gmo.jp/engineer/jisedai/blog/topic-model/)
- 例2: [Wikipedia: Topic model](https://en.wikipedia.org/wiki/Topic_model)

基本的には文書を BoW (CountVectrizor) やそれの重みを調整した TF-IDF 等の「文書単語行列」を作成し、ここから文書館類似度や単語間類似度を元に集約（≒次元削減）を試みる。文書単語行列の作成方法や次元削減方法、類似度の求め方などで様々なアルゴリズムが提案されている。ここでは (1) Bow + [LDA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html) によりトピックモデルを行い、[PyLDAvis](https://github.com/bmabey/pyLDAvis)による可視化を通してトピックを観察してみよう。

なお、トピックモデルの注意点として、トピックそのものは人手による解釈が求められる点が挙げられる。例えば先に上げた[トピックモデル入門：WikipediaをLDAモデル化してみた](https://recruit.gmo.jp/engineer/jisedai/blog/topic-model/)における図2（下図）では「政治」「スポーツ」「国際」といったトピックが並んでいるが、実際には「4-1. トピック観察」を行う必要がある。実際に観察してみよう。

## データの準備
これまで見てきたいつものやつ。

In [1]:
!curl -O https://ie.u-ryukyu.ac.jp/~tnal/2022/dm/static/r_assesment.pkl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 34834  100 34834    0     0   100k      0 --:--:-- --:--:-- --:--:--  103k


In [2]:
import collections

import numpy as np
import pandas as pd
import spacy
from wordcloud import WordCloud

nlp = spacy.load("ja_ginza")

assesment_df = pd.read_pickle('r_assesment.pkl')
assesment_df.head()

,title,grade,required,q_id,comment
0,工業数学Ⅰ,1,True,Q21 (1),特になし
1,工業数学Ⅰ,1,True,Q21 (2),正直わかりずらい。むだに間があるし。
2,工業数学Ⅰ,1,True,Q21 (2),例題を取り入れて理解しやすくしてほしい。
3,工業数学Ⅰ,1,True,Q21 (2),特になし
4,工業数学Ⅰ,1,True,Q21 (2),スライドに書く文字をもう少しわかりやすくして欲しいです。


In [3]:
# 分かち書き
poses = ['PROPN', 'NOUN', 'VERB', 'ADJ', 'ADV'] #名詞、動詞、形容詞、形容動詞

assesment_df['wakati'] = ''
for index, comment in enumerate(assesment_df['comment']):
    doc = nlp(comment)
    wakati_words = []
    for token in doc:
        if token.pos_ in poses:
            wakati_words.append(token.lemma_)
    wakati_text = ' '.join(wakati_words)
    assesment_df.at[index, 'wakati'] = wakati_text

assesment_df

,title,grade,required,q_id,comment,wakati
0,工業数学Ⅰ,1,True,Q21 (1),特になし,特に なし
1,工業数学Ⅰ,1,True,Q21 (2),正直わかりずらい。むだに間があるし。,正直 わかる ずらい むだ 間 ある
2,工業数学Ⅰ,1,True,Q21 (2),例題を取り入れて理解しやすくしてほしい。,例題 取り入れる 理解 する
3,工業数学Ⅰ,1,True,Q21 (2),特になし,特に なし
4,工業数学Ⅰ,1,True,Q21 (2),スライドに書く文字をもう少しわかりやすくして欲しいです。,スライド 書く 文字 もう 少し わかる する
...,...,...,...,...,...,...
165,データマイニング,3,False,Q22,課題が難しいものが多く、時間を多くとってもらえたのは非常に良かったですがかなりきつかったです...,課題 難しい もの 多い 時間 多い とる もらえる 非常 良い かなり きつい ござる
166,ICT実践英語Ⅰ,3,False,Q22,オンラインなどで顔を合わせてやりたかったです。,オンライン 顔 合わせる やる
167,知能情報実験Ⅲ,3,True,Q21 (2),unityの操作方法の説明などを最初に行ってもらえたらもう少しスムーズにできたのではないかと思う。,unity 操作方法 説明 最初 行く もらえる もう 少し スムーズ できる 思う
168,知能情報実験Ⅲ,3,True,Q22,それぞれに任せるといった形で進められたものだったのでそれなりに進めやすかったですが、オンライ...,それぞれ 任せる いう 形 進める もの なり 進める オンライン 班 員 指導 全く する...


## 文書ベクトルの作成
ここでは CountVectorizer (Bag-of-Words) で作成してみよう。

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

stop_words = ['こと', '\r\n', 'ため', '思う', 'いる', 'ある', 'する', 'なる']
vectorizer = CountVectorizer(stop_words=stop_words)
bow_tf_vector = vectorizer.fit_transform(assesment_df['wakati'])
print('bow_tf_vector.shape = ', bow_tf_vector.shape)

bow_tf_vector.shape =  (170, 738)


## LDAによるトピックモデル解析
sklearnでは [LatentDirichletAllocation](https://scikit-learn.org/stable/modules/decomposition.html?highlight=lda#latent-dirichlet-allocation-lda) として用意されている。

In [5]:
from sklearn.decomposition import LatentDirichletAllocation

NUM_TOPICS = 20 #トピック数
max_iter = 100  #LDAによる学習回数
lda = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=max_iter, learning_method='online',verbose=True)
data_lda = lda.fit_transform(bow_tf_vector)

iteration: 1 of max_iter: 100
iteration: 2 of max_iter: 100
iteration: 3 of max_iter: 100
iteration: 4 of max_iter: 100
iteration: 5 of max_iter: 100
iteration: 6 of max_iter: 100
iteration: 7 of max_iter: 100
iteration: 8 of max_iter: 100
iteration: 9 of max_iter: 100
iteration: 10 of max_iter: 100
iteration: 11 of max_iter: 100
iteration: 12 of max_iter: 100
iteration: 13 of max_iter: 100
iteration: 14 of max_iter: 100
iteration: 15 of max_iter: 100
iteration: 16 of max_iter: 100
iteration: 17 of max_iter: 100
iteration: 18 of max_iter: 100
iteration: 19 of max_iter: 100
iteration: 20 of max_iter: 100
iteration: 21 of max_iter: 100
iteration: 22 of max_iter: 100
iteration: 23 of max_iter: 100
iteration: 24 of max_iter: 100
iteration: 25 of max_iter: 100
iteration: 26 of max_iter: 100
iteration: 27 of max_iter: 100
iteration: 28 of max_iter: 100
iteration: 29 of max_iter: 100
iteration: 30 of max_iter: 100
iteration: 31 of max_iter: 100
iteration: 32 of max_iter: 100
iteration: 33 of 

## トピックの観察。
- [pyLDAvis](https://github.com/bmabey/pyLDAvis)によりトピックを観察してみよう。
- 下図の左側がトピック分布を表している。丸の大きさがトピック内に含まれる文書数、丸と丸の距離はトピック間の距離。
- 下図の右側が単語の発生頻度を表している。
  - トピックを選択するとそのトピックにおける単語の発生頻度を観察できる。
  - 単語を選択すると、その単語がどのようにトピック分布上にバラけているかを観察できる。

In [6]:
import pyLDAvis
from pyLDAvis import sklearn as sklearn_vis

pyLDAvis.enable_notebook()
dash = sklearn_vis.prepare(lda, bow_tf_vector, vectorizer, mds='tsne')
dash

/Users/tnal/.venv/dm/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/tnal/.venv/dm/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/Users/tnal/.venv/dm/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/Users/tnal/.venv/dm/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/Users/tnal/.venv/dm/lib/python

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
17     -10.878068   10.538728       1        1  41.804803
11     -60.884026  -75.591820       2        1  10.331622
12      -2.336119  -53.289074       3        1   8.402243
15     -61.385319  -12.914634       4        1   6.262956
19     -28.737743  126.296997       5        1   4.654199
8        1.066606 -123.552811       6        1   4.468259
7       45.140720   -7.947869       7        1   3.440363
5     -140.795975  -98.530548       8        1   3.251865
3       61.531681  -83.240562       9        1   3.157605
14     -72.563057 -140.396881      10        1   3.122404
4       15.984491   63.578186      11        1   2.675185
10    -160.851624   68.945145      12        1   1.630948
16      85.961830   51.922935      13        1   1.574962
13     -99.020836  109.936348      14        1   1.199596
2     -176.764130   -8.681684      15        1   0.928375
18    -117.189880  -37.420200      16        1   0.921461
6      108.829918  -22.946203      17        1   0.850258
1       47.588097  121.934814      18        1   0.507192
9     -106.467834   30.176889      19        1   0.462147
0      -50.613804   59.328842      20        1   0.353556, topic_info=     Term       Freq      Total Category  logprob  loglift
108    よい  18.000000  18.000000  Default  30.0000  30.0000
667    課題  31.000000  31.000000  Default  29.0000  29.0000
669    講義  30.000000  30.000000  Default  28.0000  28.0000
114   わかる  21.000000  21.000000  Default  27.0000  27.0000
437    授業  36.000000  36.000000  Default  26.0000  26.0000
..    ...        ...        ...      ...      ...      ...
375    延期   0.009928   1.111538  Topic20  -6.6038   0.9267
422    承知   0.009928   2.803466  Topic20  -6.6038   0.0016
241   入れる   0.009928   1.078309  Topic20  -6.6038   0.9571
478    最初   0.009928   6.636561  Topic20  -6.6038  -0.8601
173  メンバー   0.009928   3.081424  Topic20  -6.6038  -0.0929

[958 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
0         1  0.405249      cm
0         3  0.607873      cm
2        10  0.573755     css
3         1  0.791793  denchu
4        10  0.573756  github
...     ...       ...     ...
728       9  0.235766     面白い
731       1  0.987628      願う
732       1  0.371879      高い
732       2  0.371879      高い
734       5  0.939695      高校

[620 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[18, 12, 13, 16, 20, 9, 8, 6, 4, 15, 5, 11, 17, 14, 3, 19, 7, 2, 10, 1])